### LangChain Examples Notebook: Agents


IMPORTS

In [5]:
import torch
from ais_utils.Model_from_LC_Ollama import get_chatLLM, get_chatOpenAI, get_LLM

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
llm = get_chatOpenAI() # get_chatLLM()

In [7]:
#%pip install -qU duckduckgo-search langchain-community

In [8]:
# custom search tool
from langchain_community.tools import DuckDuckGoSearchResults

search_tool = DuckDuckGoSearchResults(backend='news')

# testing tool
search_tool.invoke("What is breaking news headlines for today")

'snippet: The Ukrainian president said the two fighters from China were detained in Donetsk, and "many more" fighting for Russia., title: Zelensky: Ukraine Captured Chinese Citizens Fighting for Russia, link: https://www.msn.com/en-us/news/world/zelensky-ukraine-captured-chinese-citizens-fighting-for-russia/ar-AA1Cwynq, date: 2025-04-08T12:37:54+00:00, source: Newsweek, snippet: The Supreme Court on Monday allowed President Donald Trump to enforce the Alien Enemies Act for now, handing the White House a significant victory that will let immigration officials rely on a sweeping wartime authority to rapidly deport alleged gang members., title: Supreme Court allows Trump to enforce Alien Enemies Act for rapid deportations for now, link: https://www.msn.com/en-us/news/politics/supreme-court-allows-trump-to-enforce-alien-enemies-act-for-rapid-deportations-for-now/ar-AA1CtIIZ, date: 2025-04-07T23:05:13+00:00, source: CNN, snippet: At least five House Republicans reportedly will not back the 

In [12]:
from langchain.tools import BaseTool

# Simple tool class (no input schema)
class CalculatorTool(BaseTool):
    name = "calculator"
    description = "Performs basic math using Python's eval function."

    def _run(self, query: str) -> str:
        return str(eval(query))

# Create an instance of the tool
calc_tool = CalculatorTool()

# Use the tool
result = calc_tool.run("2 + 3 * 4")
print("Result:", result)  # Output: 14


PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.11/u/model-field-overridden

In [6]:
from langchain.agents import Tool

# Define tools for the agents
tools = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    ),
    Tool(
        name="Calculator", 
        func=calculator_tool.run,
        description="useful for when you need to perform mathematical calculations",
    )
]

tool_names =  [tool.name for tool in tools]

In [7]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [8]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...
Sure, here's the solution:

3^5 * (12 + 3)^2 = 243 * 15 = 360

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': "Sure, here's the solution:\n\n3^5 * (12 + 3)^2 = 243 * 15 = 360"}

In [10]:
agent_executor.invoke(
    {
        "input": "Was Eines Stein born earlear than Winston Churchill?, or was it named after the German city of Wesel?"
    }
)



> Entering new AgentExecutor chain...
The context does not provide any information about the birthdate or origin of Eines Stein, so I cannot answer this question from the provided context.

> Finished chain.


{'input': 'Was Eines Stein born earlear than Winston Churchill?, or was it named after the German city of Wesel?',
 'output': 'The context does not provide any information about the birthdate or origin of Eines Stein, so I cannot answer this question from the provided context.'}